***
# Identifying the domain delimitations

In [17]:
path_project = "/media/concha-eloko/Linux/depolymerase_building"
dir_out = f"{path_project}/seekfold_phagedepo"
import pandas as pd
import os 

path_info = f"/media/concha-eloko/Linux/depolymerase_building/depolymerase_fold.csv"
info_df = pd.read_csv(path_info , sep = "\t", header = 0)
dico_folds = {"jelly-roll" : [],
              "alpha/alpha toroid" : [],
              "right-handed beta-helix" : [] ,
              "TIM beta/alpha-barrel" : [],
              "6-bladed beta-propeller" : [] ,
              "Flavodoxin-like" : [] ,
              "Alpha/Beta hydrolase fold" : [] ,
              "Other" : [],
             }

outputs = [f"{dir_out}/{file}" for file in os.listdir(dir_out) if file[-3:]=="out"]
header_seekfold = ["query","target","pident","alnlen","gapopen","qstart","qend","tstart","tend","bits","prob"]
depo_results = {}
for results in outputs :
    results_df = pd.read_csv(f"{results}", sep = "\t" , names = header_seekfold)
    for _,row in results_df.iterrows() :
        fold = info_df[info_df["ID"] == row["target"]]["Folds"].values[0]
        if fold != "jelly-roll" :
            if row["prob"] >= 0.5 :
                depo_results[row["query"].split(".pdb")[0]] = {"Fold" : fold , "Prob" : row["prob"]}
            elif fold == "right-handed beta-helix" and row["prob"] >= 0.2 :
                depo_results[row["query"].split(".pdb")[0]] = {"Fold" : fold , "Prob" : row["prob"]}

In [19]:
len(depo_results)

936

In [7]:
from tqdm import tqdm
import json
import sys
import os 
import pandas as pd 
from Bio import SeqIO

path_project = "/media/concha-eloko/Linux/depolymerase_building"
dir_out_full = f"{path_project}/seekfold_phagedepo"
dir_out = f"{path_project}/seekfold_PhageDepo_domains"
path_swords_out = f"{path_project}/sword2_PhageDepo_pt2"


path_info = f"/media/concha-eloko/Linux/depolymerase_building/depolymerase_fold.csv"
info_df = pd.read_csv(path_info , sep = "\t", header = 0)
info_df

,Activity,ID,Folds
0,xylanase,2d97.pdb,jelly-roll
1,Polysaccharide lyase 8,7fhv.pdb_A,alpha/alpha toroid
2,Polysaccharide lyase 8,7fhv.pdb_B,alpha/alpha toroid
3,Polysaccharide lyase 8,7fhy.pdb_A,alpha/alpha toroid
4,Polysaccharide lyase 8,7fhy.pdb_B,alpha/alpha toroid
...,...,...,...
61,GH5_4,4YZPA_1_1_313.pdb,TIM beta/alpha-barrel
62,GH5_8,1WKYA_1_1_302.pdb,TIM beta/alpha-barrel
63,GH5_34,2Y8KA_1_1_313.pdb,TIM beta/alpha-barrel
64,GH15,1UG9A_3_270_687.pdb,alpha/alpha toroid


In [8]:
from tqdm import tqdm
import json
import sys
import os 
import pandas as pd 
from Bio import SeqIO

path_project = "/media/concha-eloko/Linux/depolymerase_building"
dir_out_full = f"{path_project}/seekfold_phagedepo"
dir_out = f"{path_project}/seekfold_PhageDepo_domains"
path_swords_out = f"{path_project}/sword2_PhageDepo_pt2"


path_info = f"/media/concha-eloko/Linux/depolymerase_building/depolymerase_fold.csv"
info_df = pd.read_csv(path_info , sep = "\t", header = 0)


header_seekfold = ["query","target","pident","alnlen","gapopen","qstart","qend","tstart","tend","bits","prob"]
dico_folds_77 = {"jelly-roll" : [],
              "alpha/alpha toroid" : [],
              "right-handed beta-helix" : [] ,
              "TIM beta/alpha-barrel" : [],
              "6-bladed beta-propeller" : [] ,
              "Flavodoxin-like" : [] ,
              "Alpha/Beta hydrolase fold" : [] ,
              "Other" : [],
             }

errors = {'A1l_00058_A_1_129_274.pdb',
 'AF-A0A089GJA0-F1-model_v4.pdb',
 'AF-A0A4V2M274-F1-model_v4.pdb',
 'AF-A0A7W9JCS6-F1-model_v4.pdb','AF-A0A658Y395-F1-model_v4.pdb'}

dico_out = {}
for file in os.listdir(dir_out) :
    protein = file.split("_A")[0]
    if protein not in dico_out : 
        tmp_list = []
        tmp_list.append(f"{dir_out}/{file}")
        dico_out[protein] = tmp_list
    else :
        dico_out[protein].append(f"{dir_out}/{file}")

dico_domain_result = {}
for protein in tqdm(dico_out) :
    the_winner = (0,0,0)
    len_protein = len(list(SeqIO.parse(f"{path_swords_out}/{protein}/{protein}_A/SWORD/{protein}_A/{protein}_A.fasta", "fasta"))[0])
    # Collecting data for each proteins
    results_full_prot = pd.read_csv(f"{dir_out_full}/{protein}.out" , sep = "\t" , names = header_seekfold).fillna("missing")
    results_full_prot = results_full_prot[~results_full_prot["target"].isin(errors)]
    for _,row in results_full_prot.iterrows():
        fold = info_df[info_df["ID"] == row["target"]]["Folds"].values[0]
        if fold != "jelly-roll" :
            full_score = int(row["bits"])**1.2 / len_protein**0.40
            #full_score = int(row["bits"])
            target = row["target"]
            break
    the_full_prot = (full_score , f"len alignemnt : {row['alnlen']}" ,f"Full protein {protein}")
    for path in dico_out[protein] :
        if os.path.getsize(path) > 0 :
            results_df = pd.read_csv(f"{path}", sep = "\t" , names = header_seekfold).fillna("missing")
            results_df = results_df[results_df["target"] == target]
            if results_df.empty == False :
                aa_end = results_df["query"].values[0].split("_")[-1].split(".pdb")[0]
                aa_start = results_df["query"].values[0].split("_")[-2]
                PU_length = int(aa_end) - int(aa_start)
                score = int(results_df["bits"].values[0])**1.2 / PU_length**0.40
                #score = int(results_df["bits"].values[0]) 
                contender = (score , PU_length ,f"len alignemnt : {results_df['alnlen'].values[0]}",results_df["query"].values[0])
                if contender[0] > the_winner[0] :
                    the_winner = contender
                elif contender[0] == the_winner[0] :
                    if contender[1] < the_winner[1] :
                        the_winner = contender
                else :
                    pass
    #if the_full_prot[0] > (the_winner[0] + 25) :
    if the_full_prot[0] > (the_winner[0] + 2) :
        the_winner = the_full_prot 
    if the_winner[0] > 0 :
        dico_domain_result[protein] = the_winner
    
# ********************************************************
# Raw version : 
#with open(f"{path_project}/phagedepo.domains.raw.json", "w") as outfile:
#    json.dump(dico_domain_result, outfile)
    
# *****************************************************
# Fined version :
#with open(f"{path_project}/phagedepo.domains.fined.json", "w") as outfile:
#    json.dump(dico_domain_result, outfile)
    


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1033/1033 [02:55<00:00,  5.90it/s]


In [10]:
import pprint
pp = pprint.PrettyPrinter(width = 200, sort_dicts = True, compact = True)
pp.pprint(dico_domain_result)


{'1002': (4.415134916675886, 100, 'len alignemnt : 56', '1002_A_4_57_157.pdb'),
 '1013': (44.93882827133351, 216, 'len alignemnt : 279', '1013_A_9_327_543.pdb'),
 '1015': (1.993029240295908, 130, 'len alignemnt : 38', '1015_A_5_443_573.pdb'),
 '1020': (48.79529683171618, 481, 'len alignemnt : 496', '1020_A_5_233_714.pdb'),
 '1022': (19.76484678384219, 306, 'len alignemnt : 329', '1022_A_5_279_585.pdb'),
 '1035': (28.01813490328937, 437, 'len alignemnt : 552', '1035_A_4_159_596.pdb'),
 '1043': (12.058490001728813, 233, 'len alignemnt : 210', '1043_A_6_411_644.pdb'),
 '1045': (1.1031303126240861, 169, 'len alignemnt : 88', '1045_A_1_59_228.pdb'),
 '1054': (1.1539599110384477, 151, 'len alignemnt : 149', '1054_A_4_1_152.pdb'),
 '1064': (29.260094049070023, 84, 'len alignemnt : 65', '1064_A_7_125_209.pdb'),
 '1065': (61.16813640335517, 70, 'len alignemnt : 68', '1065_A_4_143_213.pdb'),
 '1083': (1.2994373024616743, 266, 'len alignemnt : 21', '1083_A_1_125_391.pdb'),
 '1091': (9.53910022557

In [20]:
len(dico_domain_result)

1029

In [12]:
path_project = "/media/concha-eloko/Linux/depolymerase_building"
dir_out = f"{path_project}/seekfold_phagedepo"
import pandas as pd
import os 

path_info = f"/media/concha-eloko/Linux/depolymerase_building/depolymerase_fold.csv"
info_df = pd.read_csv(path_info , sep = "\t", header = 0)
dico_folds = {"jelly-roll" : [],
              "alpha/alpha toroid" : [],
              "right-handed beta-helix" : [] ,
              "TIM beta/alpha-barrel" : [],
              "6-bladed beta-propeller" : [] ,
              "Flavodoxin-like" : [] ,
              "Alpha/Beta hydrolase fold" : [] ,
              "Other" : [],
             }

outputs = [f"{dir_out}/{file}" for file in os.listdir(dir_out) if file[-3:]=="out"]
header_seekfold = ["query","target","pident","alnlen","gapopen","qstart","qend","tstart","tend","bits","prob"]
depo_results = {}
for results in outputs :
    results_df = pd.read_csv(f"{results}", sep = "\t" , names = header_seekfold)
    for _,row in results_df.iterrows() :
        fold = info_df[info_df["ID"] == row["target"]]["Folds"].values[0]
        if fold != "jelly-roll" :
            if row["prob"] >= 0.5 :
                depo_results[row["query"].split(".pdb")[0]] = {"Fold" : fold , "Prob" : row["prob"]}
                break
            elif fold == "right-handed beta-helix" and row["prob"] >= 0.2 :
                depo_results[row["query"].split(".pdb")[0]] = {"Fold" : fold , "Prob" : row["prob"]}
                break
                


In [38]:
import sys
import os 
import pandas as pd 
from Bio import SeqIO
from tqdm import tqdm
import json


path_project = "/media/concha-eloko/Linux/depolymerase_building"

# *****************************************************

training_df = pd.read_csv(f"{path_project}/Phagedepo.Dataset.2007.tsv", sep = "\t", header = 0)
positive_df = training_df[training_df["Fold"] != "Negative"]


positive_df

checked_prot = [prot.split("phagedepo__")[1] for prot in positive_df["Seq_ID"] if prot.count("phage")>0]
len(checked_prot)

704

In [27]:
import sys
import os 
import pandas as pd 
from Bio import SeqIO
from tqdm import tqdm
import json


path_project = "/media/concha-eloko/Linux/depolymerase_building"

# *****************************************************
# PPT version :
phagedepo_raw = json.load(open(f"{path_project}/phagedepo.domains.raw.json"))
phagedepo_fine = json.load(open(f"{path_project}/phagedepo.domains.fined.json"))

prot_list = [prot for prot in phagedepo_raw if prot in phagedepo_fine]

phagedpo_consensus = {}
to_check = []

for protein in prot_list : 
    if (len(phagedepo_raw[protein]) == len(phagedepo_fine[protein])) and len(phagedepo_raw[protein]) == 4 :
        if phagedepo_fine[protein][3] != phagedepo_raw[protein][3] :
            to_check.append(("raw",phagedepo_raw[protein],"fine",phagedepo_fine[protein]))    
        else :
            phagedpo_consensus[protein] = phagedepo_raw[protein][3]
    elif len(phagedepo_raw[protein]) != len(phagedepo_fine[protein]) :
        to_check.append(("raw",phagedepo_raw[protein],"fine",phagedepo_fine[protein]))
    else :
        phagedpo_consensus[protein] = "full sequence"
        pass

# quick check :
to_discard = []
for prot in phagedpo_consensus :
    if prot not in depo_results :
        to_discard.append(prot)
        
dico_phagedepo_clean = {}
for prot in phagedpo_consensus :
    if prot not in to_discard :
        if phagedpo_consensus[prot] != "full sequence" : 
            len_domain = int(phagedpo_consensus[prot].split("_")[-1].split(".")[0]) - int(phagedpo_consensus[prot].split("_")[-2])
            if len_domain >= 200 :
                dico_phagedepo_clean[prot] = phagedpo_consensus[prot]
            else :
                dico_phagedepo_clean[prot] = "full sequence"
        
        
# clean version : 
with open(f"{path_project}/phagedepo.domains.clean_final.json", "w") as outfile:
    json.dump(dico_phagedepo_clean, outfile)


In [39]:
len(dico_phagedepo_clean)

657

In [50]:
2615 in checked_prot

False

In [1]:
import sys
import os 
import pandas as pd 
from Bio import SeqIO
from tqdm import tqdm
import json


path_project = "/media/concha-eloko/Linux/depolymerase_building"

# *****************************************************
# PPT version :
phagedepo_raw = json.load(open(f"{path_project}/phagedepo.domains.raw.json"))
phagedepo_fine = json.load(open(f"{path_project}/phagedepo.domains.fined.json"))

phagedpo_consensus = {}
to_check = {}
for protein in phagedepo_raw : 
    if protein in phagedepo_fine :
        if (len(phagedepo_raw[protein]) == len(phagedepo_fine[protein])) and len(phagedepo_raw[protein]) == 4 :
            if phagedepo_fine[protein][3] != phagedepo_raw[protein][3] :
                to_check.append(("raw",phagedepo_raw[protein],"fine",phagedepo_fine[protein]))    
            else :
                phagedpo_consensus[protein] = phagedepo_raw[protein][3]
        elif len(phagedepo_raw[protein]) != len(phagedepo_fine[protein]) :
            to_check.append(("raw",phagedepo_raw[protein],"fine",phagedepo_fine[protein]))
        else :
            phagedpo_consensus[protein] = "full sequence"
            pass

In [2]:
import sys
import os 
import pandas as pd 
from Bio import SeqIO
from tqdm import tqdm
import json


path_project = "/media/concha-eloko/Linux/depolymerase_building"

# *****************************************************
# PPT version :
phagedepo_raw = json.load(open(f"{path_project}/phagedepo.domains.raw.json"))
phagedepo_fine = json.load(open(f"{path_project}/phagedepo.domains.fined.json"))

phagedpo_consensus = {}
to_check = {}
for protein in phagedepo_raw : 
    if protein in phagedepo_fine :
        if (len(phagedepo_raw[protein]) == len(phagedepo_fine[protein])) and len(phagedepo_raw[protein]) == 4 :
            if phagedepo_fine[protein][3] != phagedepo_raw[protein][3] :
                to_check.append(("raw",phagedepo_raw[protein],"fine",phagedepo_fine[protein]))    
            else :
                phagedpo_consensus[protein] = phagedepo_raw[protein][3]
        elif len(phagedepo_raw[protein]) != len(phagedepo_fine[protein]) :
            to_check.append(("raw",phagedepo_raw[protein],"fine",phagedepo_fine[protein]))
        else :
            phagedpo_consensus[protein] = "full sequence"
            pass

250

In [51]:
import pprint
pp = pprint.PrettyPrinter(width = 200, sort_dicts = True, compact = True)
pp.pprint(to_check)

[('raw', [79, 438, 'len alignemnt : 486', '2615_A_3_76_514.pdb'], 'fine', [29.138600551724867, 52, 'len alignemnt : 48', '2615_A_7_76_128.pdb']),
 ('raw', [79, 610, 'len alignemnt : 533', '5725_A_3_211_821.pdb'], 'fine', [14.81319489864659, 197, 'len alignemnt : 239', '5725_A_6_291_488.pdb']),
 ('raw', [63, 351, 'len alignemnt : 416', '5680_A_4_212_563.pdb'], 'fine', [16.579337811797153, 53, 'len alignemnt : 25', '5680_A_8_212_265.pdb']),
 ('raw', [66, 471, 'len alignemnt : 474', '3735_A_5_274_745.pdb'], 'fine', [18.414674556015456, 76, 'len alignemnt : 69', '3735_A_6_274_350.pdb']),
 ('raw', [53, 'len alignemnt : 384', 'Full protein 2598'], 'fine', [14.311182176511124, 134, 'len alignemnt : 167', '2598_A_5_455_589.pdb']),
 ('raw', [56, 'len alignemnt : 387', 'Full protein 5343'], 'fine', [9.059389804740709, 279, 'len alignemnt : 331', '5343_A_3_291_570.pdb']),
 ('raw', [69, 470, 'len alignemnt : 474', '3730_A_5_274_744.pdb'], 'fine', [21.210312874962806, 76, 'len alignemnt : 69', '373

In [55]:
proteins_to_check = [prot[-1][-1].split("_")[0] for prot in to_check]

In [32]:
import sys
import os 
import pandas as pd 
from Bio import SeqIO
from tqdm import tqdm
import json


path_project = "/media/concha-eloko/Linux/depolymerase_building"
dir_out_full = f"{path_project}/seekfold_phagedepo"
dir_out = f"{path_project}/seekfold_PhageDepo_domains"
path_swords_out = f"{path_project}/sword2_PhageDepo_pt2"

wrong_smh = []
with open(f"{path_project}/Dpo_domains.phagedepo.0805.final.tsv","w") as outfile :
    outfile.write(f"Seq_ID\tFold\tProb\tBoundaries\tDpo_seq\tFull_seq\n")
    for protein,domain in phagedpo_consensus.items() :
        try : 
            fold = depo_results[protein]["Fold"]
            prob = depo_results[protein]["Prob"]
            aa_full = (list(SeqIO.parse(f"{path_swords_out}/{protein}/{protein}_A/SWORD/{protein}_A/{protein}_A.fasta", "fasta"))[0]).seq
            if domain == "full sequence" :
                outfile.write(f"phagedepo__{protein}\t{fold}\t{prob}\tfull_protein\t{aa_full}\t{aa_full}\n")
            else :
                dom_aa = (list(SeqIO.parse(f"{path_swords_out}/{protein}/{protein}_A/Protein_Units/{domain.split('.pdb')[0]}.fasta", "fasta"))[0]).seq
                outfile.write(f"phagedepo__{protein}\t{fold}\t{prob}\t{domain.split('.pdb')[0]}\t{dom_aa}\t{aa_full}\n")
        except Exception as e :
            wrong_smh.append(protein)
            #print(protein, e)


In [22]:

#phagedepo_df = pd.read_csv(f"{path_project}/Dpo_domains.phagedepo.2007.final.tsv", sep = "\t", header = 0)

In [36]:
from collections import Counter 

Counter(phagedepo_df["Fold"])

Counter({'right-handed beta-helix': 602,
         '6-bladed beta-propeller': 96,
         'TIM beta/alpha-barrel': 5,
         'Other': 1})

In [14]:
phagedepo_df

NameError: name 'phagedepo_df' is not defined